In [1]:
# Massive Comparison of different cluster algorithms
# (adpated from sklearn)
# (c) Jan Nagler

import time
import warnings
import numpy as np
import matplotlib.pyplot as plt
from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice

# Global rnd num seed: change here for quick and dirty 
np.random.seed(0)

# Number of samples
n_samples = 150

# Generate sample point clouds
noisy_circles = datasets.make_circles(n_samples=n_samples, factor=.5, noise=.05)

noisy_moons = datasets.make_moons(n_samples=n_samples, noise=.05)

blobs = datasets.make_blobs(n_samples=n_samples, random_state=8)

# Anisotropicly distributed data
random_state = 170
X, y = datasets.make_blobs(n_samples=n_samples, random_state=random_state)
transformation = [[0.6, -0.6], [-0.4, 0.8]]
X_aniso = np.dot(X, transformation)
aniso = (X_aniso, y)

# blobs with varied variances
varied = datasets.make_blobs(n_samples=n_samples,
                             cluster_std=[1.0, 2.5, 0.5],
                             random_state=random_state)

# ============
# Set up cluster parameters
# ============
plt.figure(figsize=(9 * 2 + 3, 12.5))
plt.subplots_adjust(left=.02, right=.98, bottom=.001, top=.96, wspace=.05,
                    hspace=.01)

# Default paramters (used for blobs and no_structure later)
default_base = {'quantile': .3,
                'eps': .3,
                'damping': .9,
                'preference': -200,
                'n_neighbors': 10,
                'n_clusters': 3,
                'min_samples': 20,
                'xi': 0.05,
                'min_cluster_size': 0.1}

# Actual default parameters
datasets = [
    (noisy_circles, {'damping': .77, 'preference': -240,
                     'quantile': .2, 'n_clusters': 2,
                     'min_samples': 20, 'xi': 0.25}),
    (noisy_moons, {'damping': .75, 'preference': -220, 'n_clusters': 2}),
    (varied, {'eps': .18, 'n_neighbors': 2,
              'min_samples': 5, 'xi': 0.035, 'min_cluster_size': .2}),
    (aniso, {'eps': .15, 'n_neighbors': 2,
             'min_samples': 20, 'xi': 0.1, 'min_cluster_size': .2}),
    (blobs, {})
] 

# Subplot index
plot_num = 1

# Compute and plot everything 
for i_dataset, (dataset, algo_params) in enumerate(datasets):
    
    # Set parameters with dataset-specific values
    params = default_base.copy()
    params.update(algo_params)

    # Set dataset (hidden index i_dataset)
    X, y = dataset

    # Normalize and fit
    X = StandardScaler().fit_transform(X)

    # BANDWITH method
    # Estimate bandwidth for mean shift
    bandwidth = cluster.estimate_bandwidth(X, quantile=params['quantile'])

    # WARD method
    # Connectivity matrix for structured Ward
    connectivity = kneighbors_graph(
        X, n_neighbors=params['n_neighbors'], include_self=False)
    # make connectivity symmetric
    connectivity = 0.5 * (connectivity + connectivity.T)

    # ============
    # Create cluster objects
    # ============
    
    #Mean Shift method
    ms = cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
    
    #Mini Batch K-Means 
    two_means = cluster.MiniBatchKMeans(n_clusters=params['n_clusters'])
    
    #Ward method (Hierachical clustering)
    ward = cluster.AgglomerativeClustering(
        n_clusters=params['n_clusters'], linkage='ward',
        connectivity=connectivity)
    
    #Spectal Clustering method
    spectral = cluster.SpectralClustering(
        n_clusters=params['n_clusters'], eigen_solver='arpack',
        affinity="nearest_neighbors")
    
    # DBSCAN 
    dbscan = cluster.DBSCAN(eps=params['eps'])

    #Affinity propagation method
    affinity_propagation = cluster.AffinityPropagation(
        damping=params['damping'], preference=params['preference'])
    
    #Average Linkage method (Agglomerative Clustering)
    average_linkage = cluster.AgglomerativeClustering(
        linkage="average", affinity="cityblock",
        n_clusters=params['n_clusters'], connectivity=connectivity)
    
    #Birch method (balanced iterative reducing and clustering using hierarchies)
    #(Hierarchical clustering)
    birch = cluster.Birch(n_clusters=params['n_clusters'])
    
    #Gaussian Mixure (EM method with finite number of gaussian centers)
    gmm = mixture.GaussianMixture(
        n_components=params['n_clusters'], covariance_type='full')
    
    #List of tupels with method name and pointer to it
    clustering_algorithms = (
        ('MiniBatchKMeans', two_means),
        ('Affinity', affinity_propagation),
        ('MeanShift', ms),
        ('SpectralC', spectral),
        ('Ward', ward),
        ('AggloC', average_linkage),
        ('DBSCAN', dbscan),
        ('Birch', birch),
        ('GaussMixtureEM', gmm)
    )

    # Algorithm call loop
    for name, algorithm in clustering_algorithms:
        
        # For runtime calculation
        t0 = time.time()

        #XXX Catch warnings using WITCH, related to kneighbors_graph
        #main thing: call algorithm.fit(X)
        with warnings.catch_warnings():
            warnings.filterwarnings(
                "ignore",
                message="the number of connected components of the " +
                "connectivity matrix is [0-9]{1,2}" +
                " > 1. Completing it to avoid stopping the tree early.",
                category=UserWarning)
            warnings.filterwarnings(
                "ignore",
                message="Graph is not fully connected, spectral embedding" +
                " may not work as expected.",
                category=UserWarning)
            algorithm.fit(X)
        
        # Runtime is t1-t0 now 
        t1 = time.time()
        
        #XX
        if hasattr(algorithm, 'labels_'):
            y_pred = algorithm.labels_.astype(np.int)
        else:
            y_pred = algorithm.predict(X)

        # Plot Lables and Plot Title 
        plt.subplot(len(datasets), len(clustering_algorithms), plot_num)
        if i_dataset == 0:
            plt.title(name, size=18)

        # Define colors (dynamic, cyclic, as #predicted classes)
        colors = np.array(list(islice(cycle(['r', 'g', 'b',
                                             '#ffff00', '#00ffff', '#ff00ff',
                                             '#999999', '#e41a1c', '#dede00']),
                                      int(max(y_pred) + 1))))
        
        # Add black color for outliers (if any)
        colors = np.append(colors, ["#000000"])
        plt.scatter(X[:, 0], X[:, 1], s=10, color=colors[y_pred],
                   alpha=0.3, marker='o', edgecolor='black')

        plt.xlim(-2.5, 2.5)
        plt.ylim(-2.5, 2.5)
        plt.xticks(())
        plt.yticks(())
        
        #display runtime for output
        plt.text(.99, .01, ('%.2fs' % (t1 - t0)).lstrip('0'),
                 transform=plt.gca().transAxes, size=15,
                 horizontalalignment='right')
        
        #increase subplot index
        plot_num += 1
        
# Conversion to high-resolution pdf (call this before plt.show!)
plt.savefig('aaaa_clustering_monster.pdf')

# Show entire plot figure AND save now
plt.show()

<Figure size 2100x1250 with 45 Axes>